# LangChain Academy

- https://academy.langchain.com/enrollments

## Chat models

In [2]:
## Chat models
from langchain_openai import ChatOpenAI

gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
from langchain_core.messages import HumanMessage

msg = HumanMessage(content="Hello world", name="Lance")

messages = [msg]

gpt4o_chat.invoke(messages)

In [ ]:
gpt4o_chat.invoke("hello world")

In [ ]:
gpt35_chat.invoke("hello world")

## Search Tools

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("LangGraphとはなんですか？")

In [ ]:
search_docs

## The Simplest Graph

In [9]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

In [19]:
# ノードはただの関数で定義される
# ノード内で実行したい処理を記処理を

def node_1(state):
    print("--- Node 1 ---")
    # 現在の内部状態に「私は」を追加する
    return {"graph_state": state["graph_state"] + "私は"}

def node_2(state):
    print("--- Node 2 ---")
    # 現在の内部状態に「幸せです！」を追加する
    return {"graph_state": state["graph_state"] + "幸せです！"}

def node_3(state):
    print("--- Node 3 ---")
    # 現在の内部状態に「私は悲しいです！」を追加する
    return {"graph_state": state["graph_state"] + "悲しいです！"}

In [20]:
import random
from typing import Literal


def decide_mood(state) -> Literal["node_2", "node_3"]:
    # 一般的に現在の内部状態によって次のノードを決定する
    user_input = state["graph_state"]

    # ここでは状態によらずにランダムに次のノードを選択する
    if random.random() < 0.5:
        return "node_2"
    
    return "node_3"

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({"graph_state": "こんにちは、こちらはLanceです。"})

In [ ]:
type(response)

In [ ]:
# LangGraphのinvokeの戻り値は内部状態なのか？
response

## Messages

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content="海洋哺乳類を研究していたとおっしゃいましたか？", name="Model")]
messages.append(HumanMessage(content="はい、そうです。", name="Lance"))
messages.append(AIMessage(content="素晴らしいですね。何について学びたいですか？", name="Model"))
messages.append(HumanMessage(content="アメリカで最もシャチを観察できる良い場所について学びたいです。", name="Lance"))

for m in messages:
    m.pretty_print()

## Chat Models

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
result = llm.invoke(messages)
type(result)

In [ ]:
print(result)

In [ ]:
print(result.response_metadata)

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.
    
    Args:
    a: first int
    b: second int
    """
    return a * b


llm_with_tools = llm.bind_tools([multiply])
llm_with_tools

In [ ]:
tool_call = llm_with_tools.invoke([HumanMessage(content="2かける3はなんですか？", name="Lance")])
tool_call

In [ ]:
tool_call.additional_kwargs["tool_calls"]

## messagesを状態として使う

In [32]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

## Reducers

In [4]:
from typing import Annotated
from langgraph.graph.message import add_messages


# add_messagesがReducers
# stateを更新したときに実行される関数を指定する
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [35]:
# 上記と同じ処理がかかれたMessagesStateというクラスもある
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # messagesは事前に定義されている
    pass

In [ ]:
initial_messages = [AIMessage(content="こんにちは！何かお手伝いできますか？", name="Model"),
                    HumanMessage(content="海洋生物学に関する情報を探しています。")]

new_message = AIMessage(content="はい、お手伝いできます。具体的にどのような分野に興味がありますか？")

add_messages(initial_messages, new_message)

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END


def tool_calling_llm(state: MessagesState):
    # MessagesStateを継承したグラフを使っているためこの呼び出し結果がstateのmessagesに追加される
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="こんにちは！")})
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="2かける3はなんですか？")})
for m in messages["messages"]:
    m.pretty_print()

## Router

In [40]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.
    
    Args:
    a: first int
    b: second int
    """
    return a * b


llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
from re import I
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(MessagesState)

# LLMを呼び出すノード、ツールを使うわけではない
builder.add_node("tool_calling_llm", tool_calling_llm)

# 実際にツールを使う（関数を実行する）ノード
# 関数の実行結果をToolMessageとしてmessages状態に追加する
builder.add_node("tools", ToolNode([multiply]))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm", tools_condition
)

builder.add_edge("tools", END)
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="こんにちは、世界")]
messages = graph.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="2かける3はなんですか？")]
messages = graph.invoke({"messages": messages})
# ToolNodeの出力はToolMessageで関数の戻り値のまま
for m in messages["messages"]:
    m.pretty_print()

## Agent

In [1]:
from langchain_openai import ChatOpenAI


def multiply(a: int, b: int) -> int:
    """Multiply a and b.
    
    Args:
        a: first int
        b: second int
    """
    return a * b


def add(a: int, b: int) -> int:
    """Add a and b.
     
    Args:
        a: first int
        b: second int
    """
    return a + b


def divide(a: int, b: int) -> float:
    """Divide a and b.
     
    Args:
        a: first int
        b: second int
    """
    return a / b


tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [3]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage


sys_msg = SystemMessage(content="あなたは一連の入力に対して算術計算を行うことを任務とする、役立つアシスタントです。")

def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")

# tools_conditionはツールを使うかどうかを決定する関数
# 使わない場合はENDにいく
builder.add_conditional_edges("assistant", tools_condition)

# ツールの出力をLLMに再度渡して解釈させる（Observe）
builder.add_edge("tools", "assistant")

react_graph = builder.compile()

display(Image(react_graph.get_graph().draw_mermaid_png()))

In [6]:
messages = [HumanMessage(content="3と4を足してください。その結果を2倍してください。その結果を5で割ってください。")]
messages = react_graph.invoke({"messages": messages})

In [ ]:
for m in messages["messages"]:
    m.pretty_print()

## Memory

In [ ]:
messages = [HumanMessage(content="3と4を足してください。")]
messages = react_graph.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
# 直前のグラフのinvoke結果を保持していない
messages = [HumanMessage(content="それに2をかけてください。")]
messages = react_graph.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

messages = [HumanMessage(content="3と4を足してください。")]

messages = react_graph_memory.invoke({"messages": messages}, config)
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="それに2をかけてください。")]

# 同じconfigを与えると過去の実行の履歴が引き継がれる
messages = react_graph_memory.invoke({"messages": messages}, config)
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
memory.get(config)["channel_values"]["messages"]

In [34]:
memory.get({"configurable": {"thread_id": "2"}})